In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
jobs = pd.read_csv('indeed_ml_dataset/train.tsv', sep='\t', encoding='utf-8')
stop_words = pd.read_csv('stopwords.txt', sep=" ", header = None)[0].values.tolist()

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem

english_stemmer = nltk.stem.SnowballStemmer('english', ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([english_stemmer.stem(w) for w in analyzer(doc)])

vectorizer_s = StemmedCountVectorizer(analyzer="word", stop_words=stop_words, ngram_range=(1, 4))

In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vectorizer',   vectorizer_s),
    ('classifier',         MultinomialNB())
])

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

binarizer = MultiLabelBinarizer()
y = binarizer.fit_transform(jobs.tags.str.split().fillna('other'))
print(y.shape)
print(type(y))

(4375, 17)
<class 'numpy.ndarray'>


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

params = {'estimator__C': np.logspace(-5, 5, 11)}

clf = OneVsRestClassifier(LogisticRegression())
cv = GridSearchCV(clf, params, n_jobs=-1, scoring='f1', cv=5)
bow = vectorizer_s.fit_transform(jobs['description'])
cv.fit(bow, y);